In [2]:
import os
import shutil
import warnings
import json
import glob
import nibabel as nb
import json
warnings.filterwarnings('ignore')

In [7]:
# Load main settings parameters
with open('settings.json') as f:
    json_s = f.read()
    settings = json.loads(json_s)

subjects = settings['subjects']
sessions = settings['sessions']
scratch_dir = settings['data_dir']
disk_dir = "/Users/sinakling/disks/meso_shared"

bids_dir = "{}/{}".format(disk_dir, settings['project_name'])
deriv_dir = "{}/derivatives".format(bids_dir)
source_dir = "{}/sourcedata".format(bids_dir)

end_files = ['.nii.gz','.json','.tsv']

output_dir = os.path.join(deriv_dir, "log_outputs")
os.makedirs(output_dir, exist_ok=True)
output_sh = os.path.join(output_dir, "create_folders.sh")

In [8]:
print(bids_dir)
print(deriv_dir)
print(source_dir)

/Users/sinakling/disks/meso_shared/pRF_am
/Users/sinakling/disks/meso_shared/pRF_am/derivatives
/Users/sinakling/disks/meso_shared/pRF_am/sourcedata


In [ ]:
# Map from new session names to identifying keywords in old session names
SESSION_MAP = {
    "ses-01": ["CAL"],
    "ses-02": ["pRF_strab"],
    "ses-03": ["pRF"],
    "ses-04": ["pRF_occl"],
    "ses-05": ["pRF_aniso"],
    "ses-06": ["RSL", "RSC", "RSR"],
}

project_dir = os.path.join(settings['data_dir'], settings['project_name'])
sourcedata_dir = os.path.join(project_dir, "sourcedata")
os.makedirs(sourcedata_dir, exist_ok=True)

for sub in subjects:
    print(f"\n🔹 Processing {sub}")

    sub_dir = os.path.join(project_dir, sub)
    source_backup = os.path.join(sourcedata_dir, sub)

    # 1) Copy full original subject folder to sourcedata (if not already there)
    if not os.path.exists(source_backup):
        print(f"→ Backing up {sub_dir} → {source_backup}")
        shutil.copytree(sub_dir, source_backup, dirs_exist_ok=True)
    else:
        print(f"Backup already exists for {sub}, skipping copy")

    # 2) Reorganize inside the main project dir
    occl_source = "ses-prfoccl2" if sub == "sub-002" else "ses-prfoccl"

    # Create new session folders
    for new_ses in SESSION_MAP.keys():
        new_ses_dir = os.path.join(sub_dir, new_ses)
        os.makedirs(os.path.join(new_ses_dir, "func"), exist_ok=True)
        os.makedirs(os.path.join(new_ses_dir, "fmap"), exist_ok=True)

    # Identify all original sessions
    orig_sessions = [
        s for s in os.listdir(sub_dir)
        if s.startswith("ses-") and s not in SESSION_MAP
    ]

    for new_ses, keywords in SESSION_MAP.items():
        target_func_dir = os.path.join(sub_dir, new_ses, "func")

        # Special case for sub-002 pRF_occl
        if new_ses == "ses-04" and sub == "sub-002":
            sources = [occl_source]
        else:
            sources = [
                s for s in orig_sessions
                if any(k.lower() in s.lower() for k in keywords)
            ]

        if not sources:
            print(f"No source session found for {new_ses}")
            continue

        for source in sources:
            source_func_dir = os.path.join(sub_dir, source, "func")
            if not os.path.exists(source_func_dir):
                continue

            # Copy all functional files (.nii.gz, .json, .tsv)
            for ext in [".nii.gz", ".json", ".tsv"]:
                for fpath in glob(os.path.join(source_func_dir, f"*{ext}")):
                    dest = os.path.join(target_func_dir, os.path.basename(fpath))
                    shutil.copy2(fpath, dest)
                    print(f"    Copied {os.path.basename(fpath)} → {new_ses}/func")

print("\nBackup and reorganization complete.")



NameError: name 'settings' is not defined

In [ ]:
#Step 2 
# copy event files 
# event files CAL: from https://github.com/mszinte/deepmreyecalib/tree/amsterdam-24/experiment_code/data/sub-01/ses-01/func
# event files pRF states: from https://github.com/mszinte/pRFexp/tree/pRFexp_altVision/data/sub-01

# rename files to match sessions